<a href="https://colab.research.google.com/github/AbdulHaqq2001/Database_management/blob/main/DBMS_Week9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00


In [2]:
import sqlite3
import pandas as pd
%load_ext sql

In [4]:
# Connect to SQLite database (or create it if it doesn’t exist)
conn = sqlite3.connect('lab_session.db')
c = conn.cursor()

In [22]:
import sqlite3

# Create a SQLite database and a connection
conn = sqlite3.connect("university.db")
c = conn.cursor()

# Define SQL commands
commands = [
    """
    CREATE TABLE IF NOT EXISTS students (
        student_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        major TEXT
    );
    """,
    """
    INSERT INTO students (first_name, last_name, major) VALUES ("John", "Doe", "Computer Science"),
    ("Jane", "Doe", "Mathematics"), ("Mike", "Jordan", "Physics");
    """,
    """
    CREATE TABLE IF NOT EXISTS courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT,
        department_id INTEGER,
        instructor_id INTEGER,
        FOREIGN KEY (department_id) REFERENCES departments(department_id),
        FOREIGN KEY (instructor_id) REFERENCES instructors(instructor_id)
    );
    """,
    """
    INSERT INTO courses (course_name, department_id, instructor_id) VALUES ("Database Systems", 1, 1),
    ("Data Structures", 1, 2), ("Linear Algebra", 2, 3),
    ("Quantum Mechanics", 3, 4);
    """,
    """
    CREATE TABLE IF NOT EXISTS enrollments (
        enrollment_id INTEGER PRIMARY KEY,
        student_id INTEGER,
        course_id INTEGER,
        FOREIGN KEY (student_id) REFERENCES students(student_id),
        FOREIGN KEY (course_id) REFERENCES courses(course_id)
    );
    """,
    """
    INSERT INTO enrollments (student_id, course_id) VALUES (1, 1),
    (1, 2), (2, 3), (3, 4);
    """,
    """
    CREATE TABLE IF NOT EXISTS instructors (
        instructor_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        department_id INTEGER,
        FOREIGN KEY (department_id) REFERENCES departments(department_id)
    );
    """,
    """
    INSERT INTO instructors (first_name, last_name, department_id)
    VALUES ("Robert", "Smith", 1), ("Susan", "Johnson", 2),
    ("Michael", "Brown", 3), ("Elizabeth", "Taylor", 4);
    """,
    """
    CREATE TABLE IF NOT EXISTS departments (
        department_id INTEGER PRIMARY KEY,
        department_name TEXT
    );
    """,
    """
    INSERT INTO departments (department_name) VALUES ("Computer Science"),
    ("Mathematics"), ("Physics"), ("Chemistry");
    """
]

# Execute SQL commands
for command in commands:
    c.executescript(command)

# Commit changes and close connection
conn.commit()
conn.close()


In [8]:
# Reconnect to the SQLite database with double quotation marks
conn = sqlite3.connect('university.db')

# Execute INNER JOIN query and visualize the result
inner_join_query = """
SELECT students.first_name AS student_name, courses.course_name
FROM students
INNER JOIN enrollments ON students.student_id = enrollments.student_id
INNER JOIN courses ON enrollments.course_id = courses.course_id;
"""

# Import the pandas library
import pandas as pd

# Read the query result into a DataFrame
inner_join_df = pd.read_sql_query(inner_join_query, conn)

# Print the DataFrame
print(inner_join_df)

# Close the database connection
conn.close()


  student_name        course_name
0         John   Database Systems
1         John    Data Structures
2         Jane     Linear Algebra
3         Mike  Quantum Mechanics


In [11]:
# Reconnect to the SQLite database
conn = sqlite3.connect('university.db')

# Define the emulated FULL JOIN query
full_join_query_1 = """
WITH RECURSIVE full_join AS (
    SELECT students.first_name AS student_name, courses.course_name
    FROM students
    LEFT JOIN enrollments ON students.student_id = enrollments.student_id
    LEFT JOIN courses ON enrollments.course_id = courses.course_id

    UNION ALL

    SELECT students.first_name AS student_name, courses.course_name
    FROM courses
    LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
    LEFT JOIN students ON enrollments.student_id = students.student_id
    WHERE students.student_id IS NULL
)

SELECT * FROM full_join;
"""

# Import the pandas library
import pandas as pd

# Read the query result into a DataFrame
full_join_df_1 = pd.read_sql_query(full_join_query_1, conn)

# Print the DataFrame
print(full_join_df_1)

# Close the database connection
conn.close()


  student_name        course_name
0         John   Database Systems
1         John    Data Structures
2         Jane     Linear Algebra
3         Mike  Quantum Mechanics


In [12]:
# Reconnect to the SQLite database
conn = sqlite3.connect('university.db')

# Define the aggregate function query
agg_function_example_1 = """
SELECT departments.department_name, COUNT(DISTINCT instructors.instructor_id) AS instructor_count
FROM departments
LEFT JOIN instructors ON departments.department_id = instructors.department_id
GROUP BY departments.department_name;
"""

# Import the pandas library
import pandas as pd

# Read the query result into a DataFrame
agg_df_1 = pd.read_sql_query(agg_function_example_1, conn)

# Print the DataFrame
print(agg_df_1)

# Close the database connection
conn.close()


    department_name  instructor_count
0         Chemistry                 1
1  Computer Science                 1
2       Mathematics                 1
3           Physics                 1


In [15]:
conn = sqlite3.connect('university.db')

# Define the aggregate function query
agg_function_example_2 = """
SELECT courses.course_name, COUNT(DISTINCT enrollments.student_id) AS student_count
FROM courses
LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
GROUP BY courses.course_name
HAVING student_count > 0;
"""

# Read the query result into a DataFrame
agg_df_2 = pd.read_sql_query(agg_function_example_2, conn)

# Print the DataFrame
print(agg_df_2)

conn.close()

         course_name  student_count
0    Data Structures              1
1   Database Systems              1
2     Linear Algebra              1
3  Quantum Mechanics              1


In [19]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('university.db')

# Define the SQL query to determine the number of courses in each department
num_cor_dep = """
SELECT departments.department_id, departments.department_name, COUNT(courses.course_name) AS course_count
FROM departments
JOIN courses ON departments.department_id = courses.department_id
GROUP BY departments.department_id;
"""

# Read the query result into a DataFrame
num_cor_df = pd.read_sql_query(num_cor_dep, conn)

# Print the DataFrame
print(num_cor_df)

# Close the database connection
conn.close()


   department_id   department_name  course_count
0              1  Computer Science             2
1              2       Mathematics             1
2              3           Physics             1


In [27]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('university.db')

# Define the SQL query to determine the number of courses in each department
num_inst_dep = """
SELECT departments.department_id, departments.department_name, COUNT(DISTINCT instructors.first_name) AS instructor_count
FROM departments
JOIN instructors ON departments.department_id = instructors.department_id
GROUP BY departments.department_id;
"""

# Read the query result into a DataFrame
num_inst_df = pd.read_sql_query(num_inst_dep, conn)

# Print the DataFrame
print(num_inst_df)

# Close the database connection
conn.close()


   department_id   department_name  instructor_count
0              1  Computer Science                 1
1              2       Mathematics                 1
2              3           Physics                 1
3              4         Chemistry                 1


In [33]:
# Connect to the SQLite database
conn = sqlite3.connect('university.db')

num_student_per_dep = """
SELECT d.department_id, d.department_name, COUNT(e.student_id) AS number_students
FROM departments d
JOIN courses c ON d.department_id = c.department_id
JOIN enrollments e ON c.course_id = e.course_id
GROUP BY d.department_id;
"""
num_student = pd.read_sql_query(num_student_per_dep, conn)

print(num_student)

   department_id   department_name  number_students
0              1  Computer Science                4
1              2       Mathematics                2
2              3           Physics                2
